In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [20]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN_2")
secret_value_1 = user_secrets.get_secret("wand_api_token")

In [21]:
import subprocess, textwrap, os, sys, time, json, re
print(subprocess.run(["nvidia-smi"], capture_output=True, text=True).stdout)

# Quick structured view
out = subprocess.run(
    ["nvidia-smi", "--query-gpu=index,name,memory.total,memory.free,utilization.gpu", "--format=csv,noheader"],
    capture_output=True, text=True
).stdout.strip().splitlines()

print("\nDetected GPUs:")
for line in out:
    print("  ", line)

assert len(out) >= 2, "This notebook expects Kaggle dual GPU (2x T4)."
print("\n✅ Dual GPU available.")

Sat Feb 28 10:17:14 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             13W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [22]:
# llamatelemetry v0.1.0
!pip install -q https://github.com/llamatelemetry/llamatelemetry/releases/download/v0.1.0/llamatelemetry-v0.1.0-source.tar.gz

# W&B + HF downloader + metrics helpers
!pip install -q wandb huggingface_hub requests pandas pynvml psutil

import wandb, pandas as pd, requests, psutil
print("✅ Installed.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.4/798.4 kB 12.8 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
✅ Installed.


In [23]:
import os
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

secrets = UserSecretsClient()

# W&B
os.environ["wand_api_token"] = secrets.get_secret("wand_api_token")

# HF (for GGUF download)
HF_TOKEN = secrets.get_secret("HF_TOKEN_2")
login(token=HF_TOKEN)

print("✅ Secrets loaded and HF login done.")

✅ Secrets loaded and HF login done.


In [24]:
import os, time

WANDB_PROJECT = os.environ.get("WANDB_PROJECT", "llamatelemetry-kaggle")
WANDB_ENTITY  = os.environ.get("WANDB_ENTITY", None)  # optional

run = wandb.init(
    project=WANDB_PROJECT,
    entity=WANDB_ENTITY,
    name=f"kaggle-dual-t4-llamatelemetry-{int(time.time())}",
    config={
        "stack": "llamatelemetry + llama.cpp + wandb",
        "hardware": "kaggle dual t4",
    },
)

print("✅ W&B run:", run.name)

✅ W&B run: kaggle-dual-t4-llamatelemetry-1772273855


In [25]:
from huggingface_hub import hf_hub_download

repo_id = "bartowski/Qwen2.5-3B-Instruct-GGUF"
filename = "Qwen2.5-3B-Instruct-Q4_K_M.gguf"

model_path = hf_hub_download(repo_id=repo_id, filename=filename)
print("✅ Model:", model_path)

# Log config to W&B
wandb.config.update({"repo_id": repo_id, "filename": filename}, allow_val_change=True)

✅ Model: /root/.cache/huggingface/hub/models--bartowski--Qwen2.5-3B-Instruct-GGUF/snapshots/f302c64a2269a69fb27b2f9473b362f5bb8e78d8/Qwen2.5-3B-Instruct-Q4_K_M.gguf


In [26]:
import torch
from llamatelemetry.server import ServerManager

print(f"torch.cuda.device_count() = {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")

BASE_URL = "http://127.0.0.1:8090"

server = ServerManager(server_url=BASE_URL)

# You can tweak ctx_size/batch_size for T4 stability
server.start_server(
    model_path=model_path,
    gpu_layers=99,
    tensor_split="0.5,0.5",   # <-- Dual GPU
    flash_attn=1,
    port=8090,
    host="127.0.0.1",
    ctx_size=4096,
    batch_size=256,
)

print("✅ Server running:", BASE_URL)

# Log server config
wandb.config.update({
    "server_url": BASE_URL,
    "gpu_layers": 99,
    "tensor_split": "0.5,0.5",
    "ctx_size": 4096,
    "batch_size": 256,
}, allow_val_change=True)

torch.cuda.device_count() = 2
  GPU 0: Tesla T4
  GPU 1: Tesla T4
GPU Check:
  Platform: kaggle
  GPU: Tesla T4
  Compute Capability: 7.5
  Status: ✓ Compatible
Starting llama-server...
  Executable: /usr/local/lib/python3.12/dist-packages/llamatelemetry/binaries/cuda12/llama-server
  Model: Qwen2.5-3B-Instruct-Q4_K_M.gguf
  GPU Layers: 99
  Context Size: 4096
  Server URL: http://127.0.0.1:8090
Waiting for server to be ready..... ✓ Ready in 2.0s
✅ Server running: http://127.0.0.1:8090


In [28]:
from llamatelemetry.api import LlamaCppClient
import time

client = LlamaCppClient(BASE_URL)

t0 = time.time()
resp = client.chat.completions.create(
    messages=[{"role": "user", "content": "Say hello in one short sentence."}],
    max_tokens=32,
    temperature=0.2,
)
lat_ms = (time.time() - t0) * 1000

text = resp.choices[0].message.content
print("Response:", text)
print(f"Latency: {lat_ms:.1f} ms")

wandb.log({"smoke/latency_ms": lat_ms, "smoke/ok": 1})

Response: Hello! How can I assist you today?
Latency: 219.4 ms


In [29]:
import threading
from collections import defaultdict
import pynvml

class LlamaMetricsCollector:
    """
    Collects:
      - /metrics (Prometheus text)
      - /slots (JSON)
      - NVML GPU metrics for GPU0 and GPU1
      - host CPU/RAM
    """
    def __init__(self, base_url: str):
        self.base_url = base_url.rstrip("/")
        self.metrics_history = defaultdict(list)
        self.timestamps = []
        self.slots_history = []
        self.gpu_history = []
        self.host_history = []
        self.running = False
        self.lock = threading.Lock()
        try:
            pynvml.nvmlInit()
        except Exception as e:
            print("NVML init warning:", e)

    def _parse_prom_text(self, text: str):
        metrics = {}
        for line in text.split("\n"):
            if line.startswith("#") or not line.strip():
                continue
            # Handle: metric_name value
            m = re.match(r"^([a-zA-Z_:][a-zA-Z0-9_:]*)\s+([-+eE0-9.]+)$", line.strip())
            if m:
                k, v = m.group(1), float(m.group(2))
                metrics[k] = v
        return metrics

    def fetch_server_metrics(self):
        try:
            r = requests.get(f"{self.base_url}/metrics", timeout=2)
            if r.status_code == 200:
                return self._parse_prom_text(r.text)
        except Exception:
            pass
        return {}

    def fetch_slots(self):
        try:
            r = requests.get(f"{self.base_url}/slots", timeout=2)
            if r.status_code == 200:
                return r.json()
        except Exception:
            pass
        return []

    def fetch_gpu(self, gpu_index: int):
        try:
            h = pynvml.nvmlDeviceGetHandleByIndex(gpu_index)
            util = pynvml.nvmlDeviceGetUtilizationRates(h)
            mem  = pynvml.nvmlDeviceGetMemoryInfo(h)
            temp = pynvml.nvmlDeviceGetTemperature(h, pynvml.NVML_TEMPERATURE_GPU)
            pwr  = pynvml.nvmlDeviceGetPowerUsage(h) / 1000.0
            return {
                "gpu_index": gpu_index,
                "gpu_util": float(util.gpu),
                "mem_util": float(util.memory),
                "mem_used_mb": float(mem.used / 1024**2),
                "mem_total_mb": float(mem.total / 1024**2),
                "temp_c": float(temp),
                "power_w": float(pwr),
            }
        except Exception:
            return {"gpu_index": gpu_index}

    def fetch_host(self):
        return {
            "cpu_percent": float(psutil.cpu_percent(interval=None)),
            "ram_used_gb": float(psutil.virtual_memory().used / 1024**3),
            "ram_total_gb": float(psutil.virtual_memory().total / 1024**3),
        }

    def collect_once(self):
        ts = time.time()
        m = self.fetch_server_metrics()
        slots = self.fetch_slots()
        g0 = self.fetch_gpu(0)
        g1 = self.fetch_gpu(1)
        host = self.fetch_host()

        with self.lock:
            self.timestamps.append(ts)
            for k, v in m.items():
                self.metrics_history[k].append(v)
            self.slots_history.append({
                "ts": ts,
                "num_slots": len(slots),
                "num_processing": sum(1 for s in slots if s.get("is_processing", False)),
            })
            self.gpu_history.append({"ts": ts, "gpu0": g0, "gpu1": g1})
            self.host_history.append({"ts": ts, **host})

        return ts, m, slots, g0, g1, host

    def start(self, interval_s=1.0):
        self.running = True
        def loop():
            while self.running:
                self.collect_once()
                time.sleep(interval_s)
        t = threading.Thread(target=loop, daemon=True)
        t.start()
        print(f"✅ Collector started (interval={interval_s}s)")

    def stop(self):
        self.running = False
        print("🛑 Collector stopped")

collector = LlamaMetricsCollector(BASE_URL)
collector.collect_once()
print("✅ Collector initialized")

✅ Collector initialized


In [ ]:
#Do not run this code

"""
def pick_metrics(prom: dict):
    """
    Different llama.cpp builds expose different metric names.
    We log a subset if they exist.
    """
    keys = [
        "llama_requests_total",
        "llama_tokens_total",
        "llama_tokens_per_second",
        "llama_prompt_tokens_total",
        "llama_generated_tokens_total",
        "process_resident_memory_bytes",
    ]
    out = {}
    for k in keys:
        if k in prom:
            out[f"server/{k}"] = prom[k]
    return out

def wandb_metrics_loop(interval_s=2.0):
    step = 0
    while True:
        ts, prom, slots, g0, g1, host = collector.collect_once()

        log = {}
        # GPU0
        log.update({
            "gpu0/util": g0.get("gpu_util", None),
            "gpu0/mem_used_mb": g0.get("mem_used_mb", None),
            "gpu0/mem_total_mb": g0.get("mem_total_mb", None),
            "gpu0/temp_c": g0.get("temp_c", None),
            "gpu0/power_w": g0.get("power_w", None),
        })
        # GPU1
        log.update({
            "gpu1/util": g1.get("gpu_util", None),
            "gpu1/mem_used_mb": g1.get("mem_used_mb", None),
            "gpu1/mem_total_mb": g1.get("mem_total_mb", None),
            "gpu1/temp_c": g1.get("temp_c", None),
            "gpu1/power_w": g1.get("power_w", None),
        })
        # Host
        log.update({
            "host/cpu_percent": host["cpu_percent"],
            "host/ram_used_gb": host["ram_used_gb"],
            "host/ram_total_gb": host["ram_total_gb"],
        })
        # Slots
        log.update({
            "server/slots_num": len(slots),
            "server/slots_processing": sum(1 for s in slots if s.get("is_processing", False)),
        })
        # Optional server metrics
        log.update(pick_metrics(prom))

        wandb.log(log, step=step)
        step += 1
        time.sleep(interval_s)

# Start collector + W&B streamers
collector.start(interval_s=1.0)

metrics_thread = threading.Thread(target=wandb_metrics_loop, kwargs={"interval_s": 2.0}, daemon=True)
metrics_thread.start()

print("✅ W&B metrics streaming started")

"""

In [30]:
#step 9 code

import threading, time

wandb_stop_event = threading.Event()

def wandb_metrics_loop(interval_s=2.0, stop_event=None):
    step = 0
    while True:
        if stop_event is not None and stop_event.is_set():
            break

        ts, prom, slots, g0, g1, host = collector.collect_once()

        log = {}
        # GPU0
        log.update({
            "gpu0/util": g0.get("gpu_util", None),
            "gpu0/mem_used_mb": g0.get("mem_used_mb", None),
            "gpu0/mem_total_mb": g0.get("mem_total_mb", None),
            "gpu0/temp_c": g0.get("temp_c", None),
            "gpu0/power_w": g0.get("power_w", None),
        })
        # GPU1
        log.update({
            "gpu1/util": g1.get("gpu_util", None),
            "gpu1/mem_used_mb": g1.get("mem_used_mb", None),
            "gpu1/mem_total_mb": g1.get("mem_total_mb", None),
            "gpu1/temp_c": g1.get("temp_c", None),
            "gpu1/power_w": g1.get("power_w", None),
        })
        # Host
        log.update({
            "host/cpu_percent": host["cpu_percent"],
            "host/ram_used_gb": host["ram_used_gb"],
            "host/ram_total_gb": host["ram_total_gb"],
        })
        # Slots
        log.update({
            "server/slots_num": len(slots),
            "server/slots_processing": sum(1 for s in slots if s.get("is_processing", False)),
        })
        # Optional server metrics
        log.update(pick_metrics(prom))

        # Important: Only log if run is still active
        if wandb.run is not None:
            wandb.log(log, step=step)

        step += 1
        time.sleep(interval_s)

collector.start(interval_s=1.0)

metrics_thread = threading.Thread(
    target=wandb_metrics_loop,
    kwargs={"interval_s": 2.0, "stop_event": wandb_stop_event},
    daemon=True
)
metrics_thread.start()

print("✅ W&B metrics streaming started (with stop event)")

✅ Collector started (interval=1.0s)
✅ W&B metrics streaming started (with stop event)


wandb: WARNING Tried to log to step 0 that is less than the current step 46. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


In [31]:
def estimate_tokens(s: str) -> int:
    # crude token proxy (good enough for trend charts)
    return max(1, len(s.split()))

def chat_and_log(prompt: str, max_tokens=128, temperature=0.7):
    t0 = time.time()
    resp = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        temperature=temperature,
    )
    lat_ms = (time.time() - t0) * 1000.0
    text = resp.choices[0].message.content

    tok_out = estimate_tokens(text)
    wandb.log({
        "infer/latency_ms": lat_ms,
        "infer/tokens_out_est": tok_out,
        "infer/tokens_per_sec_est": (tok_out / (lat_ms/1000.0)) if lat_ms > 0 else None,
    })
    return text, lat_ms

# Quick test
txt, lat = chat_and_log("Explain what GGUF is in 2 bullet points.", max_tokens=80, temperature=0.3)
print(txt)
print("Latency(ms):", lat)

- GGUF refers to the model format and optimized parameters used for the Generation-Guided Unified Format, which is a pre-trained language model designed for fine-tuning with generation guidance.
- It was created by Anthropic as part of their efforts to standardize large language model formats to facilitate research and development in AI.
Latency(ms): 1256.530523300171


wandb: WARNING Tried to log to step 1 that is less than the current step 47. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 2 that is less than the current step 48. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 3 that is less than the current step 49. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 4 that is less than the current step 50. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 5 that is less than the current step 51. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/

In [32]:
class LoadGenerator:
    def __init__(self, interval_s=1.5):
        self.interval_s = interval_s
        self.running = False
        self.thread = None
        self.prompts = [
            "Explain CUDA in simple terms.",
            "What is GGUF? Keep it short.",
            "Summarize LLM quantization in 3 lines.",
            "Describe FlashAttention at a high level.",
        ]

    def _loop(self):
        i = 0
        while self.running:
            prompt = self.prompts[i % len(self.prompts)]
            try:
                chat_and_log(prompt, max_tokens=120, temperature=0.7)
            except Exception as e:
                wandb.log({"infer/errors": 1})
            i += 1
            time.sleep(self.interval_s)

    def start(self):
        self.running = True
        self.thread = threading.Thread(target=self._loop, daemon=True)
        self.thread.start()
        print("✅ Load generator started")

    def stop(self):
        self.running = False
        print("🛑 Load generator stopped")

load_gen = LoadGenerator(interval_s=1.5)
load_gen.start()

✅ Load generator started


In [33]:
LATENCY_ALERT_MS = 5000  # tune this

# Run a few prompts and alert if slow
for p in ["Write a short haiku about GPUs.", "Give me 5 tips to optimize inference on T4."]:
    txt, lat = chat_and_log(p, max_tokens=120, temperature=0.6)
    print(f"\nPrompt: {p}\nLatency: {lat:.0f} ms\n{txt}")
    if lat > LATENCY_ALERT_MS:
        wandb.alert(
            title="High inference latency",
            text=f"Latency {lat:.0f}ms exceeded {LATENCY_ALERT_MS}ms on tensor_split={wandb.config.get('tensor_split')}"
        )


Prompt: Write a short haiku about GPUs.
Latency: 256 ms
Gigantic cores hum,
GPU speeds through calculations,
Power in circuits.

Prompt: Give me 5 tips to optimize inference on T4.
Latency: 1686 ms
Sure! Optimizing inference for the NVIDIA T4 GPU involves several strategies that can help improve performance and reduce latency. Here are five key tips:

1. **Batch Size Optimization**:
   - Adjust your batch size to achieve optimal throughput without compromising model accuracy. Generally, larger batches can lead to better utilization of the GPU memory but may increase the risk of out-of-memory errors if not managed properly.
   - Experiment with different batch sizes and monitor the inference time and model accuracy.

2. **Model Pruning and Quantization**:
   - Perform pruning and quantization on your model to reduce its


In [36]:
# Stop load generator
try:
    load_gen.stop()
except Exception as e:
    print("Load gen stop warning:", e)

# Stop W&B metrics thread
try:
    wandb_stop_event.set()
    if metrics_thread.is_alive():
        metrics_thread.join(timeout=5)
except Exception as e:
    print("Metrics thread stop warning:", e)

# Stop collector
try:
    collector.stop()
except Exception as e:
    print("Collector stop warning:", e)

# Stop server
try:
    server.stop_server()
except Exception as e:
    print("Server stop warning:", e)

# Finish W&B last
try:
    wandb.finish()
except Exception as e:
    print("wandb.finish warning:", e)

print("✅ Cleanup complete (clean thread shutdown)")

🛑 Load generator stopped
🛑 Collector stopped
✅ Cleanup complete (clean thread shutdown)
